In [10]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import time
from scipy.stats import rankdata
from hnsw_models import HNSW, HNSW_V2, HNSWNode
from tqdm import tqdm


In [2]:
# ------------------------ #
#    Helper Functions
# ------------------------ #

def generate_random_embeddings(num_embeddings, dim):
    """Generate random embeddings."""
    return np.random.rand(num_embeddings, dim).astype(np.float32)

def mean_reciprocal_rank(y_true, y_pred):
    """Calculate Mean Reciprocal Rank (MRR)."""
    mrrs = []
    for true_labels, pred_scores in zip(y_true, y_pred):
        ranks = rankdata(-pred_scores, method='ordinal')
        true_rank = ranks[np.argmax(true_labels)]
        mrrs.append(1.0 / true_rank)
    return np.mean(mrrs)

def recall_at_k(actual, predicted, k):
    """Calculate Recall@K."""
    actual_set = set(actual[:k])
    predicted_set = set(predicted[:k])
    return len(actual_set & predicted_set) / len(actual_set)

def precision_at_k(actual, predicted, k):
    """Calculate Precision@K."""
    actual_set = set(actual[:k])
    predicted_set = set(predicted[:k])
    return len(actual_set & predicted_set) / k

def calculate_metrics(exact_labels, hnsw_labels, K):
    """Calculate recall, precision, and MRR."""
    recall_scores = []
    precision_scores = []
    mrr_scores = []

    for i in range(len(exact_labels)):
        recall = recall_at_k(exact_labels[i], hnsw_labels[i], K)
        precision = precision_at_k(exact_labels[i], hnsw_labels[i], K)
        mrr = mean_reciprocal_rank(exact_labels[i], hnsw_labels[i])

        recall_scores.append(recall)
        precision_scores.append(precision)
        mrr_scores.append(mrr)

    average_recall = np.mean(recall_scores)
    average_precision = np.mean(precision_scores)
    average_mrr = np.mean(mrr_scores)

    return average_recall, average_precision, average_mrr


def benchmark_hnsw(hnsw_class, embeddings, test_embeddings, K):
    """Benchmark HNSW or HNSW_V2."""
    hnsw = hnsw_class(max_level=3)

    start_time_hnsw_add = time.time()
    for i, emb in enumerate(embeddings):
        hnsw.add_node(emb, label=f"emb{i}")
    hnsw_add_time = time.time() - start_time_hnsw_add

    hnsw_knn_labels = []
    start_time_hnsw_search = time.time()
    for emb in test_embeddings:
        top_k_nodes = hnsw.search_knn(emb, k=K)
        hnsw_knn_labels.append([int(node.label.replace('emb', '')) for node in top_k_nodes])
    hnsw_search_time = time.time() - start_time_hnsw_search

    return hnsw_knn_labels, hnsw_add_time, hnsw_search_time

In [3]:
# ------------------------ #
#    Main Function
# ------------------------ #

def run_benchmark(num_embeddings, embedding_dim, K):
    np.random.seed(42)
    
    """Run the full benchmark for HNSW and HNSW_V2."""
    # Generate embeddings and test embeddings
    embeddings = generate_random_embeddings(num_embeddings, embedding_dim)
    test_embeddings = generate_random_embeddings(num_embeddings, embedding_dim)

    # Perform exact KNN using scikit-learn for ground truth
    start_time_knn = time.time()
    knn_model = NearestNeighbors(n_neighbors=K, algorithm='brute', metric='euclidean')
    knn_model.fit(embeddings)
    distances, indices = knn_model.kneighbors(test_embeddings)
    knn_exact_time = time.time() - start_time_knn

    exact_knn_labels = [indices[i] for i in range(len(test_embeddings))]
    exact_knn_distances = [distances[i] for i in range(len(test_embeddings))]

    # Benchmark HNSW (V1)
    hnsw_v1_labels, hnsw_v1_add_time, hnsw_v1_search_time = benchmark_hnsw(HNSW, embeddings, test_embeddings, K)

    # Benchmark HNSW_V2
    hnsw_v2_labels, hnsw_v2_add_time, hnsw_v2_search_time = benchmark_hnsw(HNSW_V2, embeddings, test_embeddings, K)

    # Calculate metrics for HNSW (V1)
    v1_recall, v1_precision, v1_mrr = calculate_metrics(exact_knn_labels, hnsw_v1_labels, K)

    # Calculate metrics for HNSW_V2
    v2_recall, v2_precision, v2_mrr = calculate_metrics(exact_knn_labels, hnsw_v2_labels, K)

    # Store the results in a pandas DataFrame
    benchmark_df = pd.DataFrame({
        'method': ['HNSW_V1', 'HNSW_V2'],
        'num_embeddings': [num_embeddings, num_embeddings],
        'embedding_dim': [embedding_dim, embedding_dim],
        'average_recall': [v1_recall, v2_recall],
        'average_precision': [v1_precision, v2_precision],
        'average_mrr': [v1_mrr, v2_mrr],
        'add_time': [hnsw_v1_add_time, hnsw_v2_add_time],
        'search_time': [hnsw_v1_search_time, hnsw_v2_search_time],
        'knn_exact_search_time': [knn_exact_time, knn_exact_time]
    })

    return benchmark_df

In [4]:
# ------------------------ #
#    Running the Benchmark
# ------------------------ #
# Parameters
num_embeddings = 1000
embedding_dim = 128
K = 10

# Run the benchmark and display the results
benchmark_results = run_benchmark(num_embeddings, embedding_dim, K)

In [5]:
benchmark_results

,method,num_embeddings,embedding_dim,average_recall,average_precision,average_mrr,add_time,search_time,knn_exact_search_time
0,HNSW_V1,1000,128,0.0336,0.0336,1.0,0.082868,0.087959,0.039304
1,HNSW_V2,1000,128,0.2366,0.2366,1.0,0.237785,0.566206,0.039304


In [6]:
benchmark_results

,method,num_embeddings,embedding_dim,average_recall,average_precision,average_mrr,add_time,search_time,knn_exact_search_time
0,HNSW_V1,1000,128,0.0336,0.0336,1.0,0.060096,0.117145,0.077236
1,HNSW_V2,1000,128,0.2366,0.2366,1.0,0.226020,0.546387,0.077236


# Run Full Benchmark

In [17]:
# Define the grid search parameters
num_embeddings_values = [int(1e4), int(1e5), int(1e6)]  # Example values for the number of embeddings
embedding_dim_values = [128, 738, 1024]      # Example values for the dimension of embeddings
K_values = [50, 100, 200]

In [18]:
from tqdm import tqdm

# Initialize an empty DataFrame to store all results
all_benchmark_results = pd.DataFrame()

# Total number of iterations for tqdm
total_iterations = len(num_embeddings_values) * len(embedding_dim_values) * len(K_values)

# Perform grid search with progress tracking
with tqdm(total=total_iterations) as pbar:
    for num_embeddings in num_embeddings_values:
        for embedding_dim in embedding_dim_values:
            for K in K_values:
                # Print the current combination
                # print(f"Iter num_embeddings={num_embeddings}, embedding_dim={embedding_dim}, K={K}")

                # Run the benchmark for the current set of parameters
                benchmark_results = run_benchmark(num_embeddings, embedding_dim, K)

                # Concatenate the results to the main DataFrame
                all_benchmark_results = pd.concat([all_benchmark_results, benchmark_results], ignore_index=True)

                # Update the progress bar
                pbar.update(1)


 48%|████▊     | 13/27 [10:36:08<11:25:04, 2936.02s/it]


KeyboardInterrupt: 

In [19]:
all_benchmark_results.sort_values(by='average_recall', ascending=False)

,method,num_embeddings,embedding_dim,average_recall,average_precision,average_mrr,add_time,search_time,knn_exact_search_time
7,HNSW_V2,10000,738,0.101004,0.101004,1.0,24.948915,67.162294,0.875711
13,HNSW_V2,10000,1024,0.099996,0.099996,1.0,27.623407,78.783004,1.027910
1,HNSW_V2,10000,128,0.094812,0.094812,1.0,11.748826,38.476377,0.319315
15,HNSW_V2,10000,1024,0.073391,0.073391,1.0,29.371940,79.769873,1.119569
9,HNSW_V2,10000,738,0.071457,0.071457,1.0,20.987157,60.905278,0.941233
3,HNSW_V2,10000,128,0.070461,0.070461,1.0,11.844858,36.559242,0.382702
25,HNSW_V2,100000,738,0.069033,0.069033,1.0,2083.485636,9855.708013,51.246814
19,HNSW_V2,100000,128,0.057248,0.057248,1.0,1380.705236,8273.188709,10.143399
17,HNSW_V2,10000,1024,0.046599,0.046599,1.0,35.942324,94.451164,1.273007
21,HNSW_V2,100000,128,0.046166,0.046166,1.0,1262.492269,5266.561156,9.304619
